# Vergelijking waterstanden Wemerpolder met debietmeetpunt Hoge Tilweg

Op basis van metingen van het waterpeil benedenstrooms van gemaal Wemerpolder is een frequentieanalyse uitgevoerd. In dit notebook wordt de betrouwbaarheid van de gemeten waterpeilen bij gemaal Wemerpolder nader onderbouwd door vergelijking met metingen bij het debietmeetpunt Hoge Tilweg.  

Auteur: Thomas de Meij (Staatsbosbeheer)  
Datum: 14 juni 2021


## Inlezen van de meetgegevens

#### Debietmeetpunt Oude Tilweg

Op 8 juni 2021 heeft Sweco een Excelbestand aangeleverd met kwartierwaarden van waterstanden en debieten van het debietmeetpunt aan de Hoge Tilweg. De metingen in het bestand lopen van 1-8-2014 10:30 tot 30-12-2017 10:45. 

In [27]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import seaborn as sns
import scipy
import scipy.stats

%matplotlib inline
sns.set_style("whitegrid")  

datadir = r'..\02_data\hoge tilweg\\'
figdir = r'..\03_result\hoge tilweg\\'

In [6]:
fpath = r'..\01_source\data hoge tilweg\210608_debietmeetpunt_dwarsdiep.xlsx'
data = pd.read_excel(fpath,sheet_name='Blad2')

In [7]:
colnames = {'Datumtijd':'datumtijd',
    'Waterhoogte [m NAP]':'htwnap',
    'Debiet [m3/s]':'htwm3s'}
data = data.rename(columns=colnames)
data = data.set_index(keys='datumtijd',drop=True)

In [8]:
# sneak preview van de data
data.head(7)

,htwnap,htwm3s
datumtijd,,
2014-08-01 10:30:00,-0.86,0.868
2014-08-01 10:45:00,-0.86,0.702
2014-08-01 11:00:00,-0.86,0.879
2014-08-01 11:30:00,-0.86,0.786
2014-08-01 11:45:00,-0.86,0.889
2014-08-01 12:15:00,-0.86,0.769
2014-08-01 12:30:00,-0.86,0.878


In [9]:
print(f'Eerste meting {data.loc[data.index.min()]}')

Eerste meting htwnap   -0.860
htwm3s    0.868
Name: 2014-08-01 10:30:00, dtype: float64


In [10]:
print(f'Laatste meting {data.loc[data.index.max(),:]}')

Laatste meting htwnap   -0.750
htwm3s    3.027
Name: 2017-12-30 10:45:00, dtype: float64


In [11]:
# beschrijvende statistiek metingen
dstat = data.describe()
dstat

,htwnap,htwm3s
count,87727.000000,87727.000000
mean,-0.862693,1.103697
std,0.052662,1.139094
min,-1.100000,-1.370000
25%,-0.900000,0.441000
50%,-0.870000,0.761000
75%,-0.840000,1.369000
max,-0.470000,14.814000


#### Kwartierwaarden Wemerpolder  

De gemeten waterstanden van Hoge Tilweg worden in dit notebook vergeleken met de gevalideerde kwartierwaarden van gemaal Wemerpolder:

In [12]:
fpath = r'..\\02_data\\validatie waterpeilen\\wemerpolder_kwartierwaarden_gevalideerd.csv'
wmpnap = pd.read_csv(fpath,index_col='datumtijd',parse_dates=True)
wmpnap

,wmpnap
datumtijd,
2012-05-01 17:15:00,-0.82
2012-05-01 17:30:00,-0.83
2012-05-01 17:45:00,-0.82
2012-05-01 18:00:00,-0.82
2012-05-01 18:15:00,-0.83
...,...
2021-03-25 12:00:00,-0.84
2021-03-25 12:15:00,-0.84
2021-03-25 12:30:00,-0.84


## Visuele beoordeling van de metingen  

Onderstaande figuren geven een eerste indruk van de metingen van Hoge Tilweg. Bij inzoomen op de metingen is te zien dat tussen 15 mei en 15 oktober 2016 en tussen 15 september en 1 november 2017 geen betrouwbare metingen zijn verzameld. De metingen in deze perioden zijn op NoData gezet.

In [13]:
def plot_rawdata(data):
    """Plot timegraph of raw measurements"""
    fig, ax = plt.subplots(nrows=2, ncols=1)
    fig.set_size_inches(8,4) # w,h

    x = data.index.values
    y = data['htwnap'].values
    ax[0].plot(x,y,color='#8b0000')
    ax[0].set_title('waterpeil mNAP')

    y = data['htwm3s'].values
    ax[1].plot(x,y,color='#8b0000')
    ax[1].set_title(label='debiet m3/s')

    fig.tight_layout(pad=2.0)

    return fig,ax

fig,ax = plot_rawdata(data)
fpath = f'{figdir}hoge tilweg - ruwe kwartierwaarden.jpg'
fig.savefig(fpath,dpi=300,bbox_inches='tight')

In [14]:
plot_rawdata(data.loc['2016-05-01 00:00:00':'2016-10-31 23:45:00',:])
fpath = f'{figdir}hoge tilweg - periode mei - okt 2016.jpg'
fig.savefig(fpath,dpi=300,bbox_inches='tight')

In [15]:
plot_rawdata(data.loc['2017-09-01 00:00:00':'2017-11-30 23:45:00',:])
fpath = f'{figdir}hoge tilweg - periode september - november 2017.jpg'
fig.savefig(fpath,dpi=300,bbox_inches='tight')

De metingen in beide perioden worden op NaN gezet, zodat ze niet meedoen in de verschilanalyse:

In [16]:
# set values to NaN
data.loc['2016-05-16 00:00:00':'2016-10-14 23:45:00',:]=np.nan
data.loc['2017-09-16 00:00:00':'2017-10-31 23:45:00',:]=np.nan
fig,ax = plot_rawdata(data)
fpath = f'{figdir}Hoge Tilweg - aangepaste kwartierwaarden.jpg'
fig.savefig(fpath,dpi=300,bbox_inches='tight')

## Visuele beoordeling debieten  

Onderstaande figuur toont de debieten van Hoge Tilweg als functie van het waterpeil. Daaruit blijkt dat de reeks negatieve debieten omvat. De volgende grafiek toont een korte periode waarin negatieve debieten optreden. Er is steeds sprake van een kortdurende periode waarin het debiet negatief wordt. Het is onduidelijk of het water op dat moment echt stroomopwaarts stroomt, of dat er sprake is van een artefact van het meetproces.  

In [17]:
def plot_Qh(data):
    """Plot water level versus discharge"""
    fig, ax = plt.subplots(nrows=1, ncols=1)
    fig.set_size_inches(8,6) # w,h

    x = data['htwnap'].values
    y = data['htwm3s'].values

    ax.scatter(x,y,color='#8b0000',s=4)
    ax.set_xlabel('waterpeil mNAP',fontsize=14)
    ax.set_ylabel('debiet m3/s',fontsize=14)
    
    fig.tight_layout(pad=2.0)

    return fig,ax

fig,ax = plot_Qh(data)
fpath = f'{figdir}Hoge Tilweg - verband tussen waterpeil en debiet.jpg'
fig.savefig(fpath,dpi=300,bbox_inches='tight')

In [18]:
# korte periode met negatieve debieten
plot_rawdata(data.loc['2015-06-13 00:00:00':'2015-06-17 23:45:00',:])

fpath = f'{figdir}Hoge Tilweg - periode met negatieve debieten.jpg'
fig.savefig(fpath,dpi=300,bbox_inches='tight')

## Frequentieverdeling debieten  

Onderstaande figuur toont de frequentieverdeling van metingen van Hoge Tilweg:
- links : empirische frequentieverdeling van kwartierwaarden van debieten 
- midden: cumulatieve fruentieverdeling debieten
- rechts: frequentieverdeling van de logaritme van debieten en een fit van de normale verdeling op de log-getransformeerde debieten

De rechtergrafiek grafiek laat zien dat de berekende debieten van Hoge Tilweg min of meer een lognormale verdeling volgen met een gemiddelde van 0,8 m3/s en een standaardafwijking van 2,5 m3/s.  
Het berekende debiet ligt 90% van de tijd beneden 2,5 m3/s, het hoogste berekende debiet is 15 m3/s.

In [19]:
data['logQm3s']=data['htwm3s'].apply(lambda x:np.log10(x) if not np.isnan(x) else x)

In [20]:
# drie grafieken van de frequentieverdeling van debieten van Hoge Tilweg
vals = data['htwm3s'].dropna().values
logvals = data['logQm3s'].dropna().values

fig, ax = plt.subplots(nrows=1, ncols=3)
fig.set_size_inches(12,4) # w,h
fig.tight_layout()

# frequentieverdeling
m, bins, patches = ax[0].hist(vals, 40, density=1, alpha=0.95)
ax[0].set_title(label='frequentieverdeling debiet')
ax[0].set_ylabel('relatieve frequentie')
ax[0].set_xlabel('debiet (m3/s)')
ax[0].set_ylim(0,0.8)

# cumulatieve frequentieverdeling
m, bins, patches = ax[1].hist(vals, 20, density=1, alpha=0.95, cumulative=True)
ax[1].set_title(label='cumulatieve frequentieverdeling debiet', fontsize=12)
ax[1].set_xlabel('debiet (m3/s)')
ax[1].set_ylim(0,1.1)

# lognormale verdeling
m, bins, patches = ax[2].hist(logvals, 20, density=1, alpha=0.95)

mu, sigma = scipy.stats.norm.fit(logvals)
best_fit_line = scipy.stats.norm.pdf(bins, mu, sigma)
ax[2].plot(bins, best_fit_line)
ax[2].set_ylim(0,1.2)
ax[2].set_xticks([-2,-1,0,1,2])
ax[2].set_xticklabels(['0,01','0.1','1.0','10','100'])

ax[2].set_title(label='frequentieverdeling 10log debiet')
ax[2].set_xlabel('debiet (m3/s)')

fpath = f'{figdir}Hoge Tilweg - empirische frequentieverdeling debiet.jpg'
fig.savefig(fpath,dpi=300,bbox_inches='tight')

## Vergelijking van waterstanden Wemerpolder en Hoge Tilweg  

Voor de vergelijking van de gemeten waterpeilen bij gemaal Wemerpolder met Hoge Tilweg kijken we naar de kwartierwaarden op hetzelfde moment. Eerst plotten we de tijdreeksen van beide meetpunten onder elkaar en daarna een grafiek waarin de metingen van beide meetpunten in hetzelfde kwartier tegen elkaar worden uitgezet.

In [21]:
# samenvoegen kwartierwaarden Hoge Tilweg en Wemperpolder tot één tabel
levels = pd.merge(wmpnap,data['htwnap'],left_index=True,right_index=True,how='inner')
levels['delta_htw'] = levels['htwnap']-levels['wmpnap']
levels

,wmpnap,htwnap,delta_htw
datumtijd,,,
2014-08-01 10:30:00,-0.86,-0.86,0.00
2014-08-01 10:45:00,-0.87,-0.86,0.01
2014-08-01 11:00:00,-0.87,-0.86,0.01
2014-08-01 11:30:00,-0.86,-0.86,0.00
2014-08-01 11:45:00,-0.87,-0.86,0.01
...,...,...,...
2017-12-30 09:45:00,-0.81,-0.75,0.06
2017-12-30 10:00:00,-0.79,-0.75,0.04
2017-12-30 10:15:00,-0.78,-0.75,0.03


In [22]:
def plot_compare(wp):
    """Plot timegraph of two water level series"""
    fig, ax = plt.subplots(nrows=2, ncols=1)
    fig.set_size_inches(8,4) # w,h

    x = wp.index.values
    y = wp['wmpnap'].values
    ax[0].plot(x,y,color='#8b0000')
    ax[0].set_title('Wemerpolder')

    y = wp['htwnap'].values
    ax[1].plot(x,y,color='#8b0000')
    ax[1].set_title(label='Hoge Tilweg')

    fig.tight_layout(pad=2.0)

    return fig,ax

fig,ax = plot_compare(levels)
fpath = f'{figdir}tijdreeks waterpeilen kwartierwaarden.jpg'
fig.savefig(fpath,dpi=300,bbox_inches='tight')

In [23]:
fig, ax = plt.subplots(nrows=1, ncols=1)
fig.set_size_inches(8,5) # w,h
fig.tight_layout(pad=2.0)

ax.scatter(levels['wmpnap'],levels['htwnap'],s=4.,c='#8b0000') #, markersize=6, color="blue")
ax.set_xlabel('wemerpolder',fontsize=12.)
ax.set_ylabel('hoge tilweg',fontsize=12.)

min = -1.2
max = -0.4
ax.plot([min,max], [min,max], color = '#726f77')
ax.set_xlim(min,max)
ax.set_ylim(min,max)

fpath = f'{figdir}scatterplot kwartierwaarden wemerpolder en hoge tilweg.jpg'
fig.savefig(fpath,dpi=300,bbox_inches='tight')

Bovenstaande grafiek laat zien dat er een duidelijk rechtlijnig verband is tussen de gemeten waterpeilen van beide meetpunten. In de grafiek liggen vele punten op elkaar, de grafiek toont het totale bereik van de afwijkingen, zonder dat meteen zichtbaar wordt waar het zwaartepunt van de datapunten ligt. 

De figuur hieronder toont een frequentieverdeling van de verschillen en een best-fit van de normale verdeling.

In [24]:
levels['delta_htw'].describe()

count    83731.000000
mean         0.020862
std          0.018096
min         -0.050000
25%          0.010000
50%          0.020000
75%          0.030000
max          0.270000
Name: delta_htw, dtype: float64

In [26]:
mydata = levels['delta_htw'].dropna().values

# fig
fig, ax = plt.subplots(nrows=1, ncols=1)
fig.set_size_inches(8,4) # w,h
fig.tight_layout()

# frequentieverdeling
mybins = np.arange(-0.05,0.30,0.01)
m, bins, patches = ax.hist(mydata, bins=mybins, density=1, alpha=0.8)
ax.set_ylabel('relatieve frequentie (%)',fontsize=14)
ax.set_xlabel('verschil waterpeil hoge tilweg met wemerpolder (m)',fontsize=14)

mu, sigma = scipy.stats.norm.fit(mydata)
best_fit_line = scipy.stats.norm.pdf(mybins, mu, sigma)
ax.plot(bins, best_fit_line, color='#8b0000', lw=2.)
ax.set_xlim(-0.10,0.20)

fpath = f'{figdir}histogram verschillen waterpeil hoge tilweg met wemerpolder.jpg'
fig.savefig(fpath,dpi=300,bbox_inches='tight')

## Conclusies

- Er is een duidelijk rechtlijnig verband tussen de waterpeilen van Wemerpolder en Hoge Tilweg.  
- De waterstanden van Hoge Tilweg liggen tussen 5 cm beneden en 27 cm boven de waterpeilen bij Wemerpolder.
- Gemiddeld ligt het niveau van Hoge Tilweg 2 cm hoger dan Wemerpolder met een 95%-betrouwbaarheidsinterval tussen -1,5 en +5,7 cm (onder de veronderstelling dat de verschillen normaal verdeeld zijn). Dit zijn afwijkingen die naar verwachting binnen de meetonzekereid van beide meetpunten liggen.  

Vergelijking van de waterpeilen van Wemerpolder met Hoge Tilweg bevestigt de betrouwbaarheid van de gemeten waterpeilen die zijn gebruikt voor de frequentieanalyse.  

